In [1]:
import unidecode
accented_string = u'Málaga żródło Dóżź←ęq"'
# accented_string is of type 'unicode'

unaccented_string = unidecode.unidecode(accented_string)
unaccented_string

'Malaga zrodlo Dozz-eq"'

In [3]:
def remove_nonalpha(s):
    return ''.join(filter(str.isalnum, s))

def normalize(s):
    return remove_nonalpha(unidecode.unidecode(s).lower())
    
def loader(fname, cut = 0, top = float('inf')):
    data = dict()
    decodeData = {}
    with open(fname, 'r') as f:
        i=0
        for x in f:
            if not i%1000000:
                print(i)
            i+=1
            if i > top:
                break
            num, key, key2 = x.split()
            
            if int(num) > cut:
                d_key1 = normalize(key)
                d_key2 = normalize(key2)
                
                if d_key1 not in decodeData:
                    decodeData[d_key1] = set()
                    
                if d_key2 not in decodeData:
                    decodeData[d_key2] = set()
                
                decodeData[d_key1].add(remove_nonalpha(key))
                decodeData[d_key2].add(remove_nonalpha(key2))
                    
                if key not in data:
                    data[key]=dict()
                data[key][key2] = num
    return data, decodeData

In [8]:
d = loader('/pio/data/data/en_wiki_20160113/vectors-200.txt')

OSError: [Errno 512] Unknown error 512: '/pio/data/data/en_wiki_20160113/vectors-200.txt'